# Image clipping (5 min)

Satellite data, which often comes structured as a GeoTIFF if you recall from  {doc}`mod1_6_DMSP_VIIRS_data_access`, can cover large areas geospatially. It's not always required to work with the entire file, in fact, it's often preferred to work only with a smaller Area Of Interest (AOI).

In this tutorial, we're going to show how you can clip a particular satellite raster file to a specific AOI, including a geometry from a geopolitical boundary. We'll also apply this clipping to an entire ImageCollection.

So far, we've been working with DMSP-OLS annual composites. For this exercise, we'll work with the VIIRS-DNB data so you can get familiar with this source. Refer to {doc}`mod1_2_introduction_to_nighttime_light_data` for a refresher on the VIIRS-DNB dataset.

**Prerequisites:**
- Make sure you have Python, Jupyter notebooks, and `ee` and `geemap` installed and are familiar with these packages
- If not, you'll want to review: 
    - {doc}`mod1_3_getting_started_with_Python`
    - {doc}`mod1_4_introduction_to_Jupyter_notebooks`
    - {doc}`mod1_5_introduction_to_GEE`
- You may also want to try this exercise to get familiar with GEE and the Python `geemap` and `ee` libraries:
    - {doc}`mod1_7_practical_exercise-image_visualization`

**Our tasks in this exercise:**
1. Get a particular monthly composite from the VIIRS-DNB stray-light corrected image collection.
2. Clip the VIIRS-DNB image to a Geometry (a buffer around a point)
2. Clip the image to the geometry of the state of California.
3. Clip all the images in the VIIRS-DNB stray-light corrected image collection to the state of California

## Get a VIIRS-DNB monthly composite

For this exercise, we're going to work with the VIIRS-Day/Night Band (DNB) image collection that has been corrected for stray-light and is cleaned and created into monthly composites. In Google Earth Engine, this Image Collection is: `NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG`

You can read more about the data and the methodology for cleaning and compositing <a href="https://eogdata.mines.edu/download_dnb_composites.html">here</a>, and more about the process to correct for stray light in this paper {cite}`mills2013viirs`.

We'll filter on December, 2019 to get a single composite to view.

#### Initialize map, get image and add as layer

In [1]:
# import geemap and ee for our Python session
import geemap, ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# get December image, we're using the "avg_rad" band
viirs2019_12 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2019-12-01","2019-12-31").select('avg_rad').median()


# initialize our map
viirsMap = geemap.Map()
viirsMap.add_basemap('SATELLITE')
viirsMap.addLayer(viirs2019_12, {}, "VIIRS-DNB Dec 2019")

In [2]:
# display our map
viirsMap

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Clip our image to an area around Los Angeles, CA

We'll create a circular AOI using the `ee.Geometry.Point` method at lon: -118.2541, lat: 34.0469 (downtown LA) and then create a 200,000 meter (200 km) buffer around that.

In [3]:
lat = 34.0469
lon = -118.2541
aoi = ee.Geometry.Point([lon, lat]).buffer(200000);

Now, let's clip our VIIRS image to our AOI and add that as a layer by passing our `aoi` object to the `.clip()` function, like this:

In [4]:
viirs2019_12_clipped = viirs2019_12.clip(aoi)

viirsMap2 = geemap.Map(center=[lat, lon],zoom=7)
viirsMap2.add_basemap('SATELLITE')
viirsMap2.addLayer(viirs2019_12_clipped, {}, "VIIRS-DNB- Greater LA Dec 2019")
viirsMap2

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleBut…

Now we can focus our attention (for analysis or visualization) on just the area around greater LA.

## Clip an image to a geopolitical boundary: State of California

We arbitrarily chose a lat/lon point and buffer to clip our image around, but in geospatial analysis, we're often called to focus on particular geopolitical boundaries.

One example might be to use the shapefile of a country or sub-national boundary to clip our image. 

#### Importing shapefiles with `geemap`

Importing shapefiles is very easy to do with geemap. It's as simple as:

```
ee_object = geemap.shp_to_ee(<pathtomyshapefile>)
```

...which you can now add to your map object:

```
Map.addLayer(ee_object, {}, 'Layer name')
```

So if you had a shapefile, say for the state of California, you could import that as just shown, get the geomtry of the shapefile (i.e. by selecting the `geometry` field) and then clip your image to that geometry, just like we did with `aoi` earlier). 

Conveniently, Google Earth Engine actually has geometries for US States as a native FeatureCollection so we dont need to get those. These are based on the US Census TIGER files from 2016 and are located in GEE at `TIGER/2016/States`

We'll retrieve the geometry for the state of California:

In [5]:
aoi_CA = ee.FeatureCollection('TIGER/2016/States').filter(ee.Filter.eq('NAME', 'California'))

Now, just as with our previous AOI, we can clip our VIIRS Dec 2019 composite image to the entire state of California.

In [6]:
viirs2019_12_ca = viirs2019_12.clip(aoi_CA)

viirsMap3 = geemap.Map(center=[lat, lon],zoom=5)
viirsMap3.add_basemap('SATELLITE')
viirsMap3.addLayer(viirs2019_12_ca, {}, "VIIRS-DNB- California Dec 2019")
viirsMap3

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleBut…

**California! Right on, man!**

## Clip an entire ImageCollection to the State of California

So far, we've just clipped the December 2019 VIIRS-DNB composite, but we can easily apply this geometry clipping to an entire Image Collection. This would help if we wanted to create time series analysis of our focus area.

#### ImageCollection `map`

Here's it's necessary to learn about the `.map()` function of ImageCollections, which as the name suggests, maps a given function each image in the collection, with some constraints as <a href="https://developers.google.com/earth-engine/guides/ic_mapping">you can read here.</a>

Previously, we passed our geometry (e.g. `aoi_CA`) to the `.clip()` function to clip a single image. We can map a function that does this to our entire ImageCollection in Python by first defining a simple function that clips a single image to our specific AOI (California).

In [7]:
# get the entire VIIRS-DNB selection -- still only selecting the "avg_rad" band
viirsDNB = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").select('avg_rad')

# here's a function that uses the CA aoi to clip an image
def clip_func(x):
    return x.clip(aoi_CA)

# for simple functions in Python, we can also use the "lambda" convention
# this is identical to what we just defined above 
# and is nice and compact for vectorizing functions on arrays and images, etc.
clip_func = lambda x: x.clip(aoi_CA)

# in fact, by using lambda, we dont need to define this "clip_func" function at all
# we can write it directly in our "map" function, like this:
viirs_CA = viirsDNB.map(lambda x: x.clip(aoi_CA))

Now we can query this image collection for any date or series of dates to visualize layers. 

Let's look at June of 2015 and June 2020, for instance:

In [8]:
viirs_CA_2015_06 = viirs_CA.filterDate('2015-06-01','2015-06-30').median()
viirs_CA_2019_06 = viirs_CA.filterDate('2019-06-01','2019-06-30').median()

viirsMap4 = geemap.Map(center=[lat, lon],zoom=5)
viirsMap4.add_basemap('SATELLITE')
viirsMap4.addLayer(viirs_CA_2015_06, {}, "VIIRS-DNB- California June 2015")
viirsMap4.addLayer(viirs_CA_2019_06, {}, "VIIRS-DNB- California June 2019")
viirsMap4

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleBut…

#### Annual composite

We could also get an annual composite of California for all of 2019, by reducing our image collection (through the `.median()` function.

In [9]:
viirs_CA_2019 = viirs_CA.filterDate('2019-01-01','2019-12-31').median()

viirsMap5 = geemap.Map(center=[lat, lon],zoom=5)
viirsMap5.add_basemap('SATELLITE')
viirsMap5.addLayer(viirs_CA_2019, {}, "VIIRS-DNB- California 2019 (median)")
viirsMap5

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleBut…

You can also get the the difference (i.e. change) in nighttime lights by subtracting images -- and much more. We'll cover operations like this in a future tutorial, but now you know how to clip an image to a specific geometry.

## References:
```{bibliography} ../references.bib
:filter: docname in docnames
```